In [119]:
from dataclasses import dataclass
from typing import Optional, Dict, Sequence
import pandas as pd
from pathlib import Path


In [120]:
DIRECTIONS: Sequence[str] = ("NB", "SB", "EB", "WB")
MOVEMENTS: Sequence[str] = ("Left", "Thru", "Right", "U-Turns")

def parse_direction_headers(header_row: Sequence[str]) -> Dict[str, str]:
    mapping: Dict[str, str] = {}
    for value in header_row:
        if not isinstance(value, str):
            continue
        parts = [part.strip() for part in value.replace("\r", "").split("\n") if part.strip()]
        if len(parts) < 2:
            continue
        direction = parts[-1].lower()
        road = " ".join(parts[:-1])
        if direction == "northbound":
            mapping["NB"] = road
        elif direction == "southbound":
            mapping["SB"] = road
        elif direction == "eastbound":
            mapping["EB"] = road
        elif direction == "westbound":
            mapping["WB"] = road
    return mapping

def _rename_turn_columns(df: pd.DataFrame) -> pd.DataFrame:
    rename: Dict[str, str] = {}
    data_cols = df.columns[1:]
    for approach, chunk in zip(DIRECTIONS, (data_cols[i:i + 4] for i in range(0, len(data_cols), 4))):
        for movement, old in zip(MOVEMENTS, chunk):
            code = movement.split("-")[0][0]
            rename[old] = f"{approach}_{code}"
    return df.rename(columns=rename)

@dataclass
class IntersectionData:
    name: str
    count_date: pd.Timestamp
    northbound_road: str
    southbound_road: str
    eastbound_road: str
    westbound_road: str
    passenger: pd.DataFrame
    heavy: Optional[pd.DataFrame] = None
    combined: pd.DataFrame = field(init=False)

    def __post_init__(self) -> None:
        base = (
            self.passenger.set_index("Start Time")
            if "Start Time" in self.passenger.columns
            else self.passenger.copy()
        )
        if self.heavy is not None:
            hv = (
                self.heavy.set_index("Start Time")
                if "Start Time" in self.heavy.columns
                else self.heavy.copy()
            )
            hv = hv.reindex(base.index, fill_value=0)
            combined = base.add(hv, fill_value=0)
        else:
            combined = base
        self.combined = combined.reset_index()

    @classmethod
    def from_workbook(cls, source: str, *, name: str,
                      northbound_road: Optional[str] = None,
                      southbound_road: Optional[str] = None,
                      eastbound_road: Optional[str] = None,
                      westbound_road: Optional[str] = None) -> "IntersectionData":
        source_path = Path(source)

        passenger = pd.read_excel(source_path, sheet_name="Passenger Vehicles", header=9)
        heavy = pd.read_excel(source_path, sheet_name="Heavy Trucks", header=9)

        passenger = _rename_turn_columns(passenger)
        heavy = _rename_turn_columns(heavy)

        header_sheet = pd.read_excel(source_path, sheet_name="Passenger Vehicles", header=None, nrows=1)
        road_names = parse_direction_headers(header_sheet.iloc[0])

        date_sheet = pd.read_excel(source_path, header=None)
        count_date = pd.to_datetime(date_sheet.iat[1, 2]).date()

        return cls(
            name=name,
            count_date=count_date,
            northbound_road=northbound_road or road_names.get("NB", ""),
            southbound_road=southbound_road or road_names.get("SB", ""),
            eastbound_road=eastbound_road or road_names.get("EB", ""),
            westbound_road=westbound_road or road_names.get("WB", ""),
            passenger=passenger,
            heavy=heavy,
        )


In [121]:
charlotte_source = "C:/Users/malex/OneDrive - Triune Infrastructure Group/Documents/Misc. Programs/SR 544 at Charlotte Rd (EXCEL EXPORT).xlsx"
charlotte = IntersectionData.from_workbook(
    charlotte_source,
    name="SR 544 at Charlotte Rd"
)

charlotte.passenger.head()


,Start Time,NB_L,NB_T,NB_R,NB_U,SB_L,SB_T,SB_R,SB_U,EB_L,EB_T,EB_R,EB_U,WB_L,WB_T,WB_R,WB_U
0,07:00:00,9,34,10,0,19,66,3,0,11,179,20,2,19,184,19,0
1,07:15:00,13,38,8,0,34,47,2,0,11,180,15,0,21,227,17,1
2,07:30:00,14,58,10,0,25,45,6,0,10,228,17,0,24,187,19,0
3,07:45:00,19,59,13,0,37,65,6,0,19,206,14,0,38,194,24,0
4,08:00:00,14,44,17,0,48,58,5,0,12,192,16,2,21,155,11,0


In [122]:
charlotte.heavy.head()

,Start Time,NB_L,NB_T,NB_R,NB_U,SB_L,SB_T,SB_R,SB_U,EB_L,EB_T,EB_R,EB_U,WB_L,WB_T,WB_R,WB_U
0,07:00:00,1,8,3,0,1,4,0,0,1,3,1,0,2,10,1,0
1,07:15:00,1,10,2,0,0,2,0,0,2,6,4,0,3,3,0,0
2,07:30:00,2,6,3,0,0,3,0,0,0,5,3,0,1,7,0,0
3,07:45:00,0,5,3,0,1,3,0,0,0,7,2,0,3,6,0,0
4,08:00:00,1,5,2,0,1,5,0,0,0,5,1,0,4,10,2,0


In [123]:
charlotte.combined.head()

,Start Time,NB_L,NB_T,NB_R,NB_U,SB_L,SB_T,SB_R,SB_U,EB_L,EB_T,EB_R,EB_U,WB_L,WB_T,WB_R,WB_U
0,07:00:00,10,42,13,0,20,70,3,0,12,182,21,2,21,194,20,0
1,07:15:00,14,48,10,0,34,49,2,0,13,186,19,0,24,230,17,1
2,07:30:00,16,64,13,0,25,48,6,0,10,233,20,0,25,194,19,0
3,07:45:00,19,64,16,0,38,68,6,0,19,213,16,0,41,200,24,0
4,08:00:00,15,49,19,0,49,63,5,0,12,197,17,2,25,165,13,0
